In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
df = pd.read_csv('../labeled_15k.csv')
df['json'] = pd.Series([[{'Title': row.Title, 'Price': row.Price, 'Currency': row.Currency, 'Count': row.Count}] for row in df.itertuples()])
df = df[['Text', 'Title', 'Price', 'Currency', 'Count', 'Label_gpt', 'json']].reset_index(drop=True)
text_df = df[['Text', 'json', 'Label_gpt']].groupby('Text').agg({
  'json': 'sum',
  'Label_gpt': lambda x: 'invalid' if any(x == 'invalid') else 'valid'
}).reset_index()

In [4]:
(df.Label_gpt == 'valid').sum()

16098

In [3]:
text_df

,Text,json,Label_gpt
0,! новая. хлопковая рубашка оверсайз с приятной...,"[{'Title': 'хлопковая рубашка оверсайз', 'Pric...",valid
1,!!! ПОМИДОРКИ !!! в 3л Банке.... по 250 руб. К...,"[{'Title': 'Икра КРАСНАЯ 140 гр. банка', 'Pric...",valid
2,"""конфетный мячик"" 🍬🏀 цена €7😻 бесплатная доста...","[{'Title': 'конфетный мячик', 'Price': '7', 'C...",valid
3,# ТРИКО Распродажа без выбораМОДЕЛЬ ТКАНЬ ТРИК...,"[{'Title': 'Трико двух нитки качества супер', ...",invalid
4,# ТРИКО Распродажа без выбораМОДЕЛЬ ТКАНЬ ТРИК...,"[{'Title': 'Трико, двух нитки качества супер',...",valid
...,...,...,...
12031,🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍...,"[{'Title': 'Подушки пух перо размер 50Х70', 'P...",valid
12032,🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍♂️🧚‍...,"[{'Title': 'Подушки пух перо размер 70Х70', 'P...",valid
12033,🧨 продам кастомный системный блок!\nуникальнос...,"[{'Title': 'Кастомный системный блок', 'Price'...",valid
12034,🪄кофта. true religion. надевала 1-2 раза. разм...,"[{'Title': 'кофта True Religion, укороченная, ...",valid


In [4]:
np.random.seed(42)


manual_test_index = np.random.choice(text_df.index, size=100, replace=False)
manual_test_mask = np.zeros(text_df.shape[0], dtype=bool)
manual_test_mask[manual_test_index] = True

valid_mask = text_df.Label_gpt == 'valid'
train_val_mask = ~manual_test_mask & valid_mask

text_df['n_bundle'] = text_df.json.apply(len)
single_bundle_mask = text_df.n_bundle == 1

val_set_index = np.random.choice(text_df[train_val_mask].index, size=500, replace=False)
val_set_mask = np.zeros(text_df.shape[0], dtype=bool)
val_set_mask[val_set_index] = True

text_df.json = text_df.json.apply(json.dumps)
df_manual_test = text_df[manual_test_mask]
df_val_set = text_df[val_set_mask]
df_train = text_df[train_val_mask & ~val_set_mask]

len(df_manual_test), len(df_val_set), len(df_train)

(100, 500, 8811)

In [5]:
df_manual_test.to_csv('manual_test_100.csv')
df_val_set.to_csv('val_set_300_sb_valid.csv')
df_train.to_csv('train_9k_valid.csv')